In [6]:
from sklearn.preprocessing import StandardScaler
import matplotlib as mpl
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import re
import ast 
from sklearn.inspection import permutation_importance
import geopy.distance as dist

def getKilometersFromCenter(data, city):
    kilometers = np.zeros(data.shape[0])
    if city == 'copenhagen':
        center =  [55.6761, 12.5683]
    elif city == 'stockholm':
        center =  [59.3293, 18.0686]
    elif city == 'london':
        center = [51.5074, -0.1272]
    else:
        center = [52.5373, 13.3603]
    for i in range(data.shape[0]):
        # data[i, 0] = (data[i, 0]- center[0])*111
        # data[i, 1] = (data[i, 1] - center[1])*111
        # kilometers[i] = np.linalg.norm(data[i, 0]-data[i, 1])
        kilometers[i] = dist.geodesic(data[i], center).km
        if(data[i, 0] <  center[0]): 
            data[i, 0] = -dist.geodesic((data[i, 0], center[1]), center).km
        else: 
            data[i, 0] = dist.geodesic((data[i, 0], center[1]), center).km
        if(data[i, 1] <  center[1]):
            data[i, 1] = -dist.geodesic((center[0], data[i, 1]), center).km
        else:
            data[i, 1] = dist.geodesic((center[0], data[i, 1]), center).km
    return kilometers, data, center

def string_features_to_num(array):
    dictOfWords = { i : np.unique(array)[i] for i in range(0, len(np.unique(array)) ) }
    dictOfWords = {v: k for k, v in dictOfWords.items()}
    
    values = np.zeros(len(array))
    for i in range(len(array)):
        values[i] = dictOfWords.get(array[i])
        
    return values, dictOfWords

def selectPropertyType(type, X, y):
    xlist = []
    ylist = []
    for i in range(X.shape[0]):
        if(X[i,-5] == type):
            xlist.append(X[i,:])
            ylist.append(y[i])
    return np.array(xlist), np.array(ylist)

def oneHotEncodingPropertyType(type, X):
    isType = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        if(X[i,0] == type):
            isType[i] = 1.0
    return isType

def plotPredVsReal(yreal, ypred, limit):
    plt.scatter(yreal, ypred, label="pred vs true")
    plt.plot(yreal, yreal, label="y=x", c='red')
    plt.xlabel("y-real")
    plt.ylabel("y-pred")
    plt.xlim((0,limit))
    plt.ylim((0,limit))
    plt.legend()
    plt.show

def host_Since_fix(X):
    yearsSince = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        yearsSince[i] = 2022 - int(X[i].split('-')[0])
    return yearsSince

def plot_feature_importance(best_model, X, y, featureNames):
    permut = permutation_importance(best_model, X, y, scoring='r2')
    importance = permut.importances_mean
    
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(importance)), importance)
    plt.gca().set(xticks=range(len(importance)), xticklabels = featureNames)
    plt.xticks(rotation=90)
    plt.show()

def plotCity(longAndLat, y):
    heatmapColors = np.clip(y, 0, 800)
    indexes = np.argsort(y)
    # c=scaler.fit_transform(y.reshape(-1,1))
    plt.scatter(longAndLat[indexes,1], longAndLat[indexes,0], c=heatmapColors[indexes], cmap=mpl.colormaps['hot'])
    plt.colorbar()
    plt.scatter(0, 0, label='center')
    plt.legend()
    plt.show()

def amenitiesOneHot(X, amenityName):
    amenities = []
    for i in range(X.shape[0]):
        amenities.append(ast.literal_eval(X[i,-1]))

    amenity = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        for j in range(len(amenities[i])):
            if(amenities[i][j]==amenityName):
                amenity[i] = 1
    return amenity

def cleanBathroomFeature(X, y, featureNum):
    bathroomErr = []
    for i in range(X.shape[0]):
        try:
            X[i,featureNum] = float(X[i,featureNum].split()[0])
        except ValueError:
            bathroomErr.append(i)
    bathroomErr = np.array(bathroomErr)
    X = np.delete(X, bathroomErr, axis=0)
    y = np.delete(y, bathroomErr)
    return X, y

def cleanCurrency(y, city):
    exchange =1
    if city=='copenhagen': exchange = 0.1579
    elif city=='stockholm': exchange = 0.088507
    elif city=='london': exchange = 1.1594
    else: exchange = 1.0151

    for i in range(len(y)):
        y[i] = float(re.sub(",", "", (y[i][1:])))*exchange
    return y

AttributeError: Module 'scipy' has no attribute '_lib'

In [ ]:
def getCityData(city):
    data = pd.read_csv(('listings\listings_' + city + '.csv.gz'), compression='gzip')
    data = data[['accommodates','bathrooms_text','bedrooms', 'beds', 'host_since', 'review_scores_rating','review_scores_location','review_scores_value','review_scores_cleanliness','reviews_per_month','number_of_reviews','calculated_host_listings_count','minimum_nights','availability_90','property_type','room_type','amenities','latitude','longitude','price']]
    data = data.dropna()

    datanp = np.array(data)
    X = datanp[:,:-1]
    y = datanp[:,-1]

    X, y = cleanBathroomFeature(X, y, 1)
    
    y = cleanCurrency(y, city)

    X[:,-5], propertytype_dict = string_features_to_num(X[:, -5])
    X[:,-4], roomtype_dict = string_features_to_num(X[:, -4])

    # print(propertytype_dict)
    #print(list(enumerate(np.bincount(np.array(X[:, -5],dtype='int64')))))
    # X, y = selectPropertyType(propertytype_dict.get('Entire rental unit'), X, y)
    
    latitudeAndLongitude = X[:,-2:]
    kilometers, longAndLat, center = getKilometersFromCenter(latitudeAndLongitude, city)
    X = X[:,:X.shape[1]-2]

    onehotSource = X[:,-3:]
    X = X[:,:X.shape[1]-3]

    isType = oneHotEncodingPropertyType(propertytype_dict.get('Entire rental unit'), onehotSource)
    X = np.hstack((X, isType.reshape(-1,1)))
    isType = oneHotEncodingPropertyType(propertytype_dict.get('Entire condo'), onehotSource)
    X = np.hstack((X, isType.reshape(-1,1)))
    isType = oneHotEncodingPropertyType(propertytype_dict.get('Private room in rental unit'), onehotSource)
    X = np.hstack((X, isType.reshape(-1,1)))
    

    patio = amenitiesOneHot(onehotSource, 'Private patio or balcony')
    tv = amenitiesOneHot(onehotSource, 'TV')
    wifi = amenitiesOneHot(onehotSource, 'Wifi')
    bedlinens = amenitiesOneHot(onehotSource, 'Bed linens')
    breakfast = amenitiesOneHot(onehotSource, 'Breakfast')
    bathtub = amenitiesOneHot(onehotSource, 'Bathtub')
    washer = amenitiesOneHot(onehotSource, 'Washer')
    elevator = amenitiesOneHot(onehotSource, 'Elevator')
    pool = amenitiesOneHot(onehotSource, 'Pool')
    hostsince = host_Since_fix(X[:,4])

    onehot = np.hstack((tv.reshape(-1,1), patio.reshape(-1,1)))
    onehot = np.hstack((onehot, wifi.reshape(-1,1)))
    onehot = np.hstack((onehot, bedlinens.reshape(-1,1)))
    onehot = np.hstack((onehot, breakfast.reshape(-1,1)))
    onehot = np.hstack((onehot, bathtub.reshape(-1,1)))
    onehot = np.hstack((onehot, washer.reshape(-1,1)))
    onehot = np.hstack((onehot, elevator.reshape(-1,1)))
    onehot = np.hstack((onehot, pool.reshape(-1,1)))
    onehot = np.hstack((onehot, kilometers.reshape(-1,1)))
    onehot = np.hstack((onehot, longAndLat))
    onehot = np.hstack((onehot, hostsince.reshape(-1,1)))
    

    X = np.hstack((X[:,:4], X[:,5:]))
    X = np.hstack((X, onehot))

    return X, y, latitudeAndLongitude
    # plotFeatureVsPrice(X[:,8], y[:], "Number of Reviews")
    

In [ ]:
featureDict = {0: 'accommodates', 1: 'bathrooms', 2: 'bedrooms', 3: 'beds', 4: 'review_scores_rating', 5: 'review_scores_location',
6: 'review_scores_value', 7: 'review_scores_cleanliness', 8: 'reviews_per_month', 9: 'number_of_reviews', 10: 'calculated_host_listings_count', 
11: 'minimum_nights', 12: 'availability_90', 13: 'Entire rental unit', 14: 'Entire condo', 15: 'Private room in rental unit', 16: 'Private patio or balcony', 
17: 'TV', 18: 'Wifi', 19: 'Bed linens', 20: 'Breakfast', 21: 'Bathtub', 22: 'Washer', 23: 'Elevator', 24: 'Pool', 25: 'Km_from_center', 26: 'km_from_latitude', 
27: 'km_from_longitude', 28: 'Host_years'}


featureNames = np.array(list(featureDict.items()))
featureNames = featureNames[:,1]

In [ ]:
lX, ly, _ = getCityData('london')
cX, cy, _ = getCityData('copenhagen')
sX, sy, _ = getCityData('stockholm')
bX, by, _ = getCityData('berlin')

print(np.mean(cy), np.mean(sy), np.mean(ly), np.mean(by))

cX.shape

In [ ]:
# Neural network on copenhagen
# cx = np.concatenate((cX,lX), axis=0)
# cY = np.concatenate((cy,ly), axis=0)
cx = np.asarray(cX).astype('float32')
cY = np.asarray(cy).astype('float32')



cX_train, cX_testval, cy_train, cy_testval = train_test_split(cx, cY, test_size=0.20, random_state=4)
cX_test, cX_val, cy_test, cy_val = train_test_split(cX_testval, cy_testval, test_size=0.5, random_state=4)

print(cX_train.shape, cX_test.shape, cX_val.shape)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
#normalize our data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler()
cx_train_sc = X_scaler.fit_transform(cX_train)
cx_val_sc = X_scaler.transform(cX_val)
cx_test_sc = X_scaler.transform(cX_test)

cx_train_sc.shape

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adagrad
#adagrad = 0.02, sgd = 
lr = 0.06
p = 0.2
model = Sequential()
#found 16 to be optimal
model.add(Dense(units=16, input_dim=29, kernel_initializer='normal', activation='relu'))
# p = 0.2 optimal
model.add(Dropout(p))
model.add(Dense(units=64, kernel_initializer='normal', activation='relu'))
# model.add(Dropout(p))
# model.add(Dense(units=16, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, activation='linear', kernel_initializer='normal'))
print(model.summary())
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_msle'
                        , patience = 3
                        , verbose = 1
                        , mode = 'auto'
                        , min_delta = 0.001)

num_epochs = 100

#larger --> increase stockholm score
#smaller --> increase copen score
# too large / too small, predictions converge to same value
b_size = 32
#optimizers: adam, sgd, 
model.compile(loss="msle", optimizer=SGD(learning_rate=lr), metrics = ['msle'])

history = model.fit(cx_train_sc, cy_train,
                   batch_size = b_size,
                   epochs = num_epochs,
                   verbose = 1,
                   validation_data=(cx_val_sc, cy_val),
                   callbacks = [callback])

# def getAccuracy(x,y):
#     predictions = model.predict(x)
#     #APE (absolute percent error)
#     APE = 100*(abs(y-predictions)/y)
#     accuracy = 100-np.mean(APE)
#     return accuracy


# accuracy = getAccuracy(cx_test_sc, cy_test)
# print('Accuracy of model on test set: ', accuracy)

# sx, sY = np.asarray(sX).astype('float32'), np.asarray(sy).astype('float32')
# acc_stock = getAccuracy(sx,sY)
# print('Model evaluated on Stockholm: ', acc_stock)

import matplotlib.pyplot as plt
plt.plot(history.history['msle'])
plt.plot(history.history['val_msle'])
plt.ylabel('msle')
plt.xlabel('epoch')
plt.title('Neural Network')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
c_pred = model.predict(cx_test_sc)
print("Copenhagen test set")
plotPredVsReal(cy_test, c_pred, 750)
print("Test r2:", round(r2_score(cy_test, c_pred),4))

In [ ]:
sx, sY = np.asarray(sX).astype('float32'), np.asarray(sy).astype('float32')
sx_sc = X_scaler.transform(sx)
s_pred = model.predict(sx_sc)
print("Stockholm evaluated on copenhagen test set")
plotPredVsReal(sY, s_pred, 750)
print("Test r2:", round(r2_score(sY, s_pred),4))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['msle'])
plt.plot(history.history['val_msle'])
plt.ylabel('msle')
plt.xlabel('epoch')
plt.title('Neural Network')
plt.legend(['train', 'validation'])
plt.show()